<a href="https://colab.research.google.com/github/DKN-11/Net-flow-method/blob/main/Net_flow_method_for_3_objective_functions_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

def compute_preference_matrix(data, maximize):
    num_solutions, num_objectives = data.shape
    preference_matrix = np.zeros((num_solutions, num_solutions))

    for obj in range(num_objectives):
        obj_values = data[:, obj]
        max_val, min_val = np.max(obj_values), np.min(obj_values)

        for i in range(num_solutions):
            for j in range(num_solutions):
                if i != j:
                    if maximize[obj]:
                        preference_matrix[i, j] += (obj_values[i] - obj_values[j]) / (max_val - min_val)
                    else:
                        preference_matrix[i, j] += (obj_values[j] - obj_values[i]) / (max_val - min_val)

    return preference_matrix

def compute_net_flow(preference_matrix):
    positive_flow = np.sum(preference_matrix, axis=1)
    negative_flow = np.sum(preference_matrix, axis=0)
    net_flow = positive_flow - negative_flow
    return positive_flow, negative_flow, net_flow

def main():
    # Read data from Excel file
    file_path = '/content/final results april.xlsx'  # Replace with your file path
    sheet_name = 'prob 7 New idea'  # Replace with your sheet name
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=[0,1,2]) # Read the first 3 columns

    # Convert DataFrame to NumPy array
    data = df.values

    # Define which objectives should be maximized (True) and minimized (False)
    maximize = [True, False, False]  # Assuming the objectives are in the same order as the columns

    # Compute preference matrix
    preference_matrix = compute_preference_matrix(data, maximize)

    # Compute net flow values
    positive_flow, negative_flow, net_flow = compute_net_flow(preference_matrix)

    # Print results
    solutions = [f"S{i+1}" for i in range(data.shape[0])]  # Generate solution labels
    print("\nPreference Matrix:")
    print(preference_matrix)
    print("\nSolution Rankings:")
    for i, sol in enumerate(solutions):
        print(f"{sol}: Φ⁺ = {positive_flow[i]:.2f}, Φ⁻ = {negative_flow[i]:.2f}, Φ = {net_flow[i]:.2f}")

    # Select the best solution (highest Net Flow Score)
    best_solution_index = np.argmax(net_flow)  # Get the index of the best solution
    best_solution = solutions[best_solution_index]
    best_solution_values = data[best_solution_index]  # Get values for the best solution

    print(f"\nBest solution: {best_solution} (Highest Net Flow Score)")
    print(f"Values of the best solution: ", end="")
    for val in best_solution_values:
        print(f"{val:.8f}", end=" ")  # Print each value with 8 decimal places
    print()  # Print a newline

if __name__ == "__main__":
    main()


Preference Matrix:
[[ 0.         -0.15028191 -0.11124568 ... -0.08569131 -0.41454442
  -0.45262804]
 [ 0.15028191  0.          0.03903624 ...  0.0645906  -0.26426251
  -0.30234613]
 [ 0.11124568 -0.03903624  0.         ...  0.02555436 -0.30329875
  -0.34138236]
 ...
 [ 0.08569131 -0.0645906  -0.02555436 ...  0.         -0.32885311
  -0.36693672]
 [ 0.41454442  0.26426251  0.30329875 ...  0.32885311  0.
  -0.03808361]
 [ 0.45262804  0.30234613  0.34138236 ...  0.36693672  0.03808361
   0.        ]]

Solution Rankings:
S1: Φ⁺ = -17.60, Φ⁻ = 17.60, Φ = -35.19
S2: Φ⁺ = -2.57, Φ⁻ = 2.57, Φ = -5.14
S3: Φ⁺ = -6.47, Φ⁻ = 6.47, Φ = -12.95
S4: Φ⁺ = -39.50, Φ⁻ = 39.50, Φ = -79.00
S5: Φ⁺ = -5.59, Φ⁻ = 5.59, Φ = -11.17
S6: Φ⁺ = 13.61, Φ⁻ = -13.61, Φ = 27.21
S7: Φ⁺ = -63.89, Φ⁻ = 63.89, Φ = -127.78
S8: Φ⁺ = 26.93, Φ⁻ = -26.93, Φ = 53.85
S9: Φ⁺ = 21.09, Φ⁻ = -21.09, Φ = 42.18
S10: Φ⁺ = -29.92, Φ⁻ = 29.92, Φ = -59.85
S11: Φ⁺ = -23.17, Φ⁻ = 23.17, Φ = -46.35
S12: Φ⁺ = 22.00, Φ⁻ = -22.00, Φ = 44.01
S13

In [ ]:
import pandas as pd

# Step 1: Specify your file and sheet
file_path = '/content/Prob 6 R,T,C.xlsx'   # 🔁 Replace with your actual file path
sheet_name = 'Sheet3'            # 🔁 Replace with your actual sheet name

# Step 2: Specify which columns to read
solution_column = 'Solution'     # 🔁 Column that identifies each solution
objectives = ['ROP', 'TWR', 'COST']   # 🔁 Objective columns you want to include
maximize = [True, False, False]           # 🔁 Set True for maximization, False for minimization (must match order)

# Step 3: Read only the specified columns from Excel
columns_to_read = [solution_column] + objectives
df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=columns_to_read)
df.set_index(solution_column, inplace=True)

# Step 4: Normalize the objectives
def normalize(df, objectives, maximize):
    normalized_df = df.copy()
    for obj, is_max in zip(objectives, maximize):
        if is_max:
            normalized_df[obj] = (df[obj] - df[obj].min()) / (df[obj].max() - df[obj].min())
        else:
            normalized_df[obj] = (df[obj].max() - df[obj]) / (df[obj].max() - df[obj].min())
    return normalized_df

normalized_df = normalize(df, objectives, maximize)

# Step 5: Compute Net Flow Scores
solutions = normalized_df.index
net_flows = {}

for sol1 in solutions:
    score = 0
    for sol2 in solutions:
        if sol1 == sol2:
            continue
        diff = normalized_df.loc[sol1, objectives] - normalized_df.loc[sol2, objectives]
        score += diff.sum()
    net_flows[sol1] = score

# Step 6: Join results
net_flow_df = pd.DataFrame({'Net Flow Score': net_flows})
final_df = normalized_df.join(net_flow_df)
final_df = final_df.sort_values(by='Net Flow Score', ascending=False)

# Step 7: Output the result
print("\nNet Flow Results:")
print(final_df)

best_solution = final_df.index[0]
print(f"\n✅ Best solution based on Net Flow Method: **{best_solution}**")



Net Flow Results:
               ROP       TWR      COST  Net Flow Score
Solution                                              
20        0.526677  0.857307  0.993632       14.241041
49        0.572072  0.818393  0.985383       14.152655
35        0.503788  0.874173  0.996246       14.070553
5         0.499336  0.877459  0.996784       14.039114
38        0.597998  0.792655  0.978941       13.839943
27        0.452683  0.908148  0.999921       13.397816
4         0.475326  0.889687  0.995555       13.388608
31        0.448553  0.910588  1.000000       13.317275
34        0.643557  0.744331  0.966913       13.100258
43        0.620646  0.762575  0.969039       12.973192
45        0.572935  0.802198  0.975030       12.868410
25        0.676269  0.704318  0.956158       12.197465
32        0.415730  0.924445  0.995832       12.160588
46        0.650310  0.722894  0.956845       11.862726
29        0.385358  0.934649  0.988998       10.810437
9         0.350780  0.955620  0.989456       1

In [ ]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of solutions
num_solutions = 100

# Generate synthetic data for each objective:
# Rop (maximization): range between 50 to 100
# MSE (minimization): range between 0.1 to 5
# Bit Life (maximization): range between 1000 to 5000

rop = np.random.uniform(50, 100, num_solutions)
mse = np.random.uniform(, 5, num_solutions)
bit_life = np.random.uniform(1000, 5000, num_solutions)

# Combine into a DataFrame
df = pd.DataFrame({
    'Rop': rop,
    'MSE': mse,
    'Bit Life': bit_life
})

# Normalize data for Net Flow method (maximize all, so invert MSE)
df_norm = df.copy()
df_norm['MSE'] = df['MSE'].max() - df['MSE']
df_norm = (df_norm - df_norm.min()) / (df_norm.max() - df_norm.min())

# Calculate net flow score: sum of normalized values
df['Net Flow Score'] = df_norm.sum(axis=1)

# Identify the best solution (highest net flow score)
best_solution_index = df['Net Flow Score'].idxmax()
best_solution = df.loc[best_solution_index]

# Save to Excel
excel_filename = "synthetic_pareto_dataset.xlsx"
df.to_excel(excel_filename, index=False)

print("Best solution index:", best_solution_index)
print("Best solution:\n", best_solution)


Best solution index: 52
Best solution:
 Rop                 96.974947
MSE                  0.809985
Bit Life          4404.546686
Net Flow Score       2.669733
Name: 52, dtype: float64


In [ ]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of solutions
num_solutions = 100

# Generate synthetic data for each objective:
# Rop (maximization): range between 50 to 100
# MSE (minimization): range between 0.1 to 5
# Bit Life (maximization): range between 1000 to 5000

rop = np.random.uniform(50, 100, num_solutions)
mse = np.random.uniform(, 5, num_solutions)
bit_life = np.random.uniform(1000, 5000, num_solutions)

# Combine into a DataFrame
df = pd.DataFrame({
    'Rop': rop,
    'MSE': mse,
    'Bit Life': bit_life
})

# Normalize data for Net Flow method (maximize all, so invert MSE)
df_norm = df.copy()
df_norm['MSE'] = df['MSE'].max() - df['MSE']
df_norm = (df_norm - df_norm.min()) / (df_norm.max() - df_norm.min())

# Calculate net flow score: sum of normalized values
df['Net Flow Score'] = df_norm.sum(axis=1)

# Identify the best solution (highest net flow score)
best_solution_index = df['Net Flow Score'].idxmax()
best_solution = df.loc[best_solution_index]

# Save to Excel
excel_filename = "synthetic_pareto_dataset.xlsx"
df.to_excel(excel_filename, index=False)

print("Best solution index:", best_solution_index)
print("Best solution:\n", best_solution)


In [ ]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Number of solutions
num_solutions = 100

# Generate synthetic data for each objective:
# Rop (maximization): range between 50 to 100
# MSE (minimization): range between 0.1 to 5
# Bit Life (maximization): range between 1000 to 5000

rop = np.random.uniform(50, 100, num_solutions)
mse = np.random.uniform(, 5, num_solutions)
bit_life = np.random.uniform(1000, 5000, num_solutions)

# Combine into a DataFrame
df = pd.DataFrame({
    'Rop': rop,
    'MSE': mse,
    'Bit Life': bit_life
})

# Normalize data for Net Flow method (maximize all, so invert MSE)
df_norm = df.copy()
df_norm['MSE'] = df['MSE'].max() - df['MSE']
df_norm = (df_norm - df_norm.min()) / (df_norm.max() - df_norm.min())

# Calculate net flow score: sum of normalized values
df['Net Flow Score'] = df_norm.sum(axis=1)

# Identify the best solution (highest net flow score)
best_solution_index = df['Net Flow Score'].idxmax()
best_solution = df.loc[best_solution_index]

# Save to Excel
excel_filename = "synthetic_pareto_dataset.xlsx"
df.to_excel(excel_filename, index=False)

print("Best solution index:", best_solution_index)
print("Best solution:\n", best_solution)
